#소프트맥스 회귀 구현하기


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

In [2]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [3]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


one hot encoding 해야하므로 y의 차원은 8x3이어야 한다.

In [5]:
y_onehot = torch.zeros(8,3)
y_onehot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_onehot.shape)
print(y_onehot)

torch.Size([8, 3])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


x의 차원은 8x4, y의 차원은 8x3이므로 w의 차원은 4x3이어야 한다.

In [6]:
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros((1,3), requires_grad=True)

최적화 함수로는 gradient descent 사용

In [7]:
#optimizer
optimizer = optim.SGD([W,b], lr=0.1)

In [8]:
epochs = 1000
for e in range(epochs+1):
  hypothesis = F.softmax(x_train.matmul(W)+b, dim=1)
  cost = (y_onehot * -torch.log(hypothesis)).sum(dim=1).mean()
  #위 코드를 z = x_train.matmul(W)+b
  #cost = F.cross_entropy(z, y_train)으로 더 쉽게 구현할 수 있다

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if e%100==0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            e, epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.704199
Epoch  200/1000 Cost: 0.622999
Epoch  300/1000 Cost: 0.565717
Epoch  400/1000 Cost: 0.515291
Epoch  500/1000 Cost: 0.467661
Epoch  600/1000 Cost: 0.421278
Epoch  700/1000 Cost: 0.375402
Epoch  800/1000 Cost: 0.329766
Epoch  900/1000 Cost: 0.285072
Epoch 1000/1000 Cost: 0.248155


#nn.Module로 구현할 수 있다.


In [ ]:
model = nn.Linear(4,3) #-> W1x1 + W2x2 + W3x3 + W4x4 + b를 만들어준다.
#-> 4개의 feature, 3개의 class
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

#학습
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))